<a href="https://colab.research.google.com/github/eliyanto29/Fuzzy-Sentiment-Analisis/blob/master/Fuzzy_Sentiment_Analysis_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Fuzzy Sentiment Analysis with an RNN**
*Mathematics Department, Universitas Ahmad Dahlan, Indonesia*

https://math.uad.ac.id/

**Author**  : Sugiyarto, Ph.D dan Joko Eliyanto, S.Si

**Email**  : joko1400015006@webmail.uad.ac.id

### Abstraksi


Sentimen analisis adalah salah satu bagian dari natural language processing. Analisis sentimen dapat dilakukan dengan berbasis kamus, atau berbasis machine learning. Analisis sentimen berbasis machine learning memiliki kelebihan kedinamisan untuk bertemu dengan dataset bahasa baru atau kosa kata baru.  Analisis sentimen berusaha untuk memahami sentimen yang terkandung dalam suatu kalimat. Sebuah kalimat bisa memiliki sentimen positif, netral atau negatif. Meskipun demikian, faktanya setiap kalimat tidak selalu bersentimen positif, negatif, atau netral secara jelas. Kami berusaha mengembangkan metode sentimen analisis yang dapat menunjukkan derajat sentimen suatu kalimat. Sentimen analisis fuzzy recurrent neural network diperkenalkan dalam makalah ini untuk menghasilkan hasil analisis sentimen yang lebih akurat. Recurrent neural network adalah metode machine learning yang popular untuk sentimen analisis. Konsep himpunan fuzzy digunakan untuk menyatakan derajat sentimen sebuah kalimat. Analisis jarak euclidean untuk menentukan kedekatan dua buah vektor digunakan untuk menunjukkan bahwa metode ini lebih baik dari metode standar. Metode yang kami ajukan berhasil menghasilkan sebuah nilai yang menunjukkan derajat sentimen sebuah kalimat. Perbandingan jarak euclid antara hasil sentimen analisis standar dan metode kami menunjukkan bahwa hasil fuzzy recurrent neural network memiliki jarak yang relatif dekat dengan nilai sentimen sesungguhnya. Sentimen analisis fuzzy recurrent neural network terbukti dapat menghasilkan hasil sentimen analisis yang lebih halus daripada metode standar.

### Import Modul

In [ ]:
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from google.colab import drive

from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

### Mengakses dataset

Dataset Tweet Airline

Dataset ini berisi tweet terhadap pelayanan @USAmericanairlines. Kita diminta mengklasifikasikan tweet positif, negatif, dan netral, diikuti dengan mengkategorikan alasan negatif (seperti "penerbangan terlambat" atau "layanan kasar").


Membuka dataset

In [ ]:
#Membuka akses ke google drive
drive.mount('/content/gdrive')

In [ ]:
#Mengakses dataset
data = pd.read_csv('/content/gdrive/My Drive/SA DATASET/Tweets.csv')
data = data.sample(frac=1).reset_index(drop=True)
print(data.shape)
data.head()

In [ ]:
#Menghapus semua kolom kecuali kolom airline_sentiment dan teks.
data = data[['airline_sentiment', 'text']]
data.head()

### Data Eksplorasi

In [ ]:
#Menghitung jumlah sentiment
data['airline_sentiment'].value_counts().sort_index().plot.bar()

In [ ]:
#Menghitung jumlah teks
data['text'].str.len().plot.hist()

### Data Cleaning

In [ ]:
#Menghapus kata @VirginAmerica
data['text'] = data['text'].str.replace('@VirginAmerica', '')
data.head()

In [ ]:
#Menghapus karakter yang tidak diinginkan
data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x))
data['text'].head()

### Data Preparation

In [ ]:
#Tokenisasi kata-kata
tokenizer = Tokenizer(num_words=5000, split=" ")
tokenizer.fit_on_texts(data['text'].values)

X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X, maxlen=200, padding='post')
X

In [ ]:
#Mengkodekan sentimen negatif=[1 0 0], neutral=[0 1 0], dan positif=[0 0 1]
y = pd.get_dummies(data['airline_sentiment']).values
[print(data['airline_sentiment'][i], y[i]) for i in range(0,5)]

In [ ]:
#Memecah dataset menjadi data latih dan data uji
#test_size: perbandingan data uji(0.2) dan data latih(0.8)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Mendefinisikan Model RNN

In [ ]:
#Mendefinisikan model RNN
model = Sequential()
model.add(Embedding(5000, 256, input_length=X.shape[1]))
model.add(Dropout(0.3))
model.add(LSTM(256, return_sequences=True, dropout=0.3, recurrent_dropout=0.2))
model.add(LSTM(256, dropout=0.3, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

In [ ]:
#Meng-compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
#Melatih Model
batch_size = 32
epochs = 2

history = model.fit(X_train, 
                    y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    verbose=2)

In [ ]:
#Plot Akurasi dan Loss
import matplotlib.pyplot as plt
acc = history.history['accuracy']
loss = history.history['loss']
epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, loss, 'b', label='Training loss')
plt.title('Akurasi dan Loss Training')
plt.legend()

plt.show()

### Sentimen Fuzzy

In [ ]:
#Fungsi sentimen fuzzy
def single_fuzzy_sentiment_biner(txt):
  x=[txt]
  print(x)
  x=tokenizer.texts_to_sequences(x)
  x=pad_sequences(x, maxlen=200, padding='post')
  predictions=model.predict(x)
  probability=predictions[0].tolist()
  fuzzy_sentiment=0.5-((1/2)*probability[0])+((1/2)*probability[1])
  return fuzzy_sentiment

In [ ]:
#Menjalankan fungsi sentimen fuzzy
print(data.text[3])
print('Sentimen Fuzzy:')
single_fuzzy_sentiment_biner(data.text[3])

In [ ]:
#Fungsi uji sentimen kalimat(positif atau negatif)[STANDAR]
def sentimen_analysis(x):
  x=[x]
  x=token.texts_to_sequences(x)
  x=pad_sequences(x, maxlen=200, padding='post')
  return model.predict_classes(x)

In [ ]:
#Menjalankan fungsi sentimen fuzzy dan standar
data.text[3]
print(x)
print('Sentimen')
print(data.airline_sentiment[3])
print('Sentimen analisis dengan CNN')
print(sentimen_analysis(x))
print('Fuzzy sentimen analisis dengan CNN')
single_fuzzy_sentiment_biner(x)

### Evaluasi Sentimen Fuzzy

In [ ]:
#Mendefinisikan fungsi fuzzy sentimen pada satu kolom(banyak kalimat)
def fuzzy_sentimen_column(bahan):
  fuzzy_sentimen_kolom=[]
  for kalimat in bahan:
    fuzzy_sentimen_kolom.append(single_fuzzy_sentiment_biner(kalimat))
  return fuzzy_sentimen_kolom

In [ ]:
#Mendefinisikan fungsi uji sentimen pada satu kolom(banyak kalimat)
def sentimen_column(bahan):
  x=token.texts_to_sequences(bahan)
  x=pad_sequences(x, maxlen=input_length, padding='post')
  word_sentimen=model.predict_classes(x)
  word_sentimen=gabung_sublist(word_sentimen)
  return word_sentimen

In [ ]:
#Fungsi menggabung sublist
def gabung_sublist(list1):
  list2=[]
  for i in list1:
    list2.append(float(i))
  return list2

In [ ]:
#Mendefinisikan pengurangan dua list
def substract_two_list(list1,list2):
  difference = []
  zip_object = zip(list1, list2)
  for list1_i, list2_i in zip_object:
    difference.append(list1_i-list2_i)
  return difference

In [ ]:
#Menghitung jarak euclid vektor sentimen(asli) dan prediksi CNN dan prediksi fuzzy sentimen
bahan=data.text
hasil1=fuzzy_sentimen_column(bahan)
squared_hasil1 = [number ** 2 for number in hasil1]
print(squared_hasil1)
hasil2=sentimen_column(bahan)
squared_hasil2 = [number ** 2 for number in hasil2]
print(squared_hasil2)
sentimencoba=imdb_data.sentiment[:100]
squared_sentimencoba = [number ** 2 for number in sentimencoba]
print(squared_sentimencoba)
print(abs(sum(substract_two_list(squared_hasil1,squared_sentimencoba))))
print(abs(sum(substract_two_list(squared_hasil2,squared_sentimencoba))))

### Kesimpulan

Fuzzy Sentimen Analisis menyajikan sentimen sebuah kalimat dalam bentuk fuzzy atau derajat sentimen sebuah kalimat(tidak pasti[0 1])

**Akhir dokumen**

---